In [1]:


def initialSetup():
    global con
    global cur
    con= sqlite3.connect('projects.db')
    cur= con.cursor()


    # creating tables
    cur.execute('''CREATE TABLE IF NOT EXISTS Project (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL UNIQUE, 
        project_name TEXT  NOT NULL,
        description TEXT,
        priority TEXT CHECK(priority in ('low', 'medium', 'high')) NOT NULL DEFAULT 'low'
        )
     ''')

    cur.execute('''CREATE TABLE IF NOT EXISTS Employee  (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL UNIQUE, 
        name TEXT NOT NULL,
        age INTEGER NOT NULL,
        salary REAL NOT NULL
        )''')

    cur.execute('''CREATE TABLE IF NOT EXISTS Project_Employee  (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL UNIQUE, 
        project_id INTEGER NOT NULL,
        employee_id INTEGER NOT NULL)
        ''')





In [2]:
class EmployeeNotFound(Exception):
    pass
class ProjectNotFound(Exception):
    pass

In [3]:
def getText(message, error, minChars):
    data= input(message)
    while len(data) < minChars:
        print(error)
        data= input(message)
    return data

def getPriority():
    while True:
        try:
            priority= int(input('Select a priority for the project\n1. Low\n2.Medium\n3. High\nEnter your choice'))
            if priority==  1:
                return  'low'
            elif priority ==2:
                return 'medium'
            elif priority ==3:
                return 'high'
            else:
                print('Invalid priority')
        except:
            print('Invalid priority')
            
            
def getAge():
    
    while True:
        age= getNumber('Enter age of the employee: ', 'Age should be a number.')
        if age>20 and age <=65:
            return age
        else:
            print('Invalid age. Age must be between 20 and 65')
                
        
def getNumber(message, error):
    while True:
        try:
            return int(input(message))
        except:
            print(error)
        
def getSalary():
    while True:
        try:
            salary= float(input('Enter annual salary of the employee'))
            if salary> 36000:
                return salary
            else:
                print('The salary should be atleast 3000 a month.')
                
        except:
            print('Salary should be a number')
def getMenuChoice():
    while True:
        try:
            print('Menu'.center(30,'*'))
            print('1. Create Employee')
            print('2. Create Project')
            print('3. Add Employee to the project')
            print('4. Display All Employees')
            print('5. Display All Projects')
            print('6. Display Project Members')
            print('7. Display Projects of an employee')
            print('8. Delete an employee ')
            print('9. Delete a Project')
            print('0. Quit')
            print('*'*30)
            choice= int(input('Enter your choice:'))
            if 0<=choice<=9:
                return choice
            else:
                print('Invalid choice. Choice should be between 0 and 9')
        except:
            print('Your choice sould be an integer.')


In [4]:
def createEmployee():
    name= getText("Enter name of the employee: ", 'Name of the employee must be atleast 5 letters long',5)
    age= getAge()
    salary= getSalary()
    sql= '''INSERT INTO Employee (name, age, salary) values(?,?,?)'''
    cur.execute(sql, (name,age, salary))
    con.commit()
    print('Employee Created Succefully.')

    
def createProject():
    name= getText("Enter name of the Project: ", 'Project name must be atleast 5 letters long', 5)
    description= getText("Enter Project Description: ", 'Project description must not be less than 15 characters.', 15)
    priority= getPriority()
    sql= '''INSERT INTO Project (project_name, description, priority) values(?,?,?)'''
    cur.execute(sql, (name,description, priority))
    con.commit()
    print('Project Created Succefully.')
    
    
def filterByName(tableName):
    name= getText("Enter name of the "+tableName+"(partial name also works): ", 'The name should not be empty', 1)
    if tableName== 'Project':
        sql = '''SELECT * FROM Project WHERE project_name LIKE ?'''
    else:
        sql = '''SELECT * FROM Employee WHERE name LIKE ?'''
    pattern ='%'+name+'%'
    print(pattern)
    data= con.execute(sql, (pattern,)).fetchall()
    
    return data
def filterById(tableName, data):
    for row in data:
        print(row)
    print('There are multiple data values that matches with the '+tableName.lower()+' name. Please select the id from the list') 
    id= getNumber('Enter '+tableName+' id', 'id should be an integer')
    if tableName== 'Project':
        sql= '''select * from Project where id=?'''
    else:
        sql= '''select * from Employee where id=?'''
    data= cur.execute(sql, (id,)).fetchone()
    return data
    
def getEmployee():
    data= filterByName('Employee')
    if data== None:
        raise EmployeeNotFound('Employee with the specified id does not exist')
    if len(data)>1:
        employee= filterById('Employee', data)
        if employee== None:
            raise ProjectNotFound('Employee with the specified id does not exist')
        if(len(employee) == 0):
            raise EmployeeNotFound('Employee with the specified id does not exist')
        else:
            return employee
    elif len(data)== 1:
        return data[0]
    else:
        raise EmployeeNotFound("Unable to find the employee")
def getProject():
    data= filterByName('Project')
    if data== None:
        raise ProjectNotFound('Project with the specified id does not exist')
    if len(data)>1:
        project= filterById('Project', data)
        if project== None:
            raise ProjectNotFound('Project with the specified id does not exist')
        if(len(project) == 0):
            raise ProjectNotFound('Project with the specified id does not exist')
        else:
            return project
    elif len(data)== 1:
        return data[0]
    else:
        raise ProjectNotFound("Unable to find the project")
def addEmployeeToProject():
    project= getProject()
    print("Selected Project: "+ str(project))
    employee= getEmployee()
    print("Selected Employee: "+ str(employee))
    cur.execute(sql, (project[0],employee[0]))
    con.commit()
    print('Added project to employee!')
    
#     project= filterByName('Project')
#     sql= '''insert into Project_Employee (project_id, employee_id) values(?,?)'''
#     if len(project)>1:
#         project= filterById('Project', project)
#         if(len(project) == 0):
#             print('Project with the specified id does not exist')
#         else:
#             employee= getEmployee()
           
#             cur.execute(sql, (project[0],employee[0]))
#             con.commit()
#             print('Added project to employee!')
#     elif len(project)==1:
#         employee= getEmployee()
#         cur.execute(sql, (project[0][0],employee[0]))
#         con.commit()
#         print('Added project to employee!')
#     else:
#         print('Project with the specified name does not exist')
        
def getEmployeeForProject(projectId):
    sql= '''select * from Employee join Project_Employee on Employee.id=Project_Employee.employee_id where project_id= ?'''
    data= cur.execute(sql,(projectId,)).fetchall()
    return data
    

def displayAll(tableName):
    if tableName== 'Employee':
        
        sql= '''select * from Employee'''
    else:
        sql= '''select * from Project'''
    data= cur.execute(sql).fetchall()
    if len(data)!=0:
        for row in data:
            print(row)
    else:
        print("No",tableName.lower(),'exists')
def displayProjectMembers():
    project= getProject()
    print("Selected Project: "+ str(project))
    employees= getEmployeeForProject(project[0])
    if len(employees)== 0:
        print('No employees for this project')
    else:
        for employee in employees:
            print(employee[0:4])
    
    
#     project= filterByName('Project')
#     if len(project)>1:
#         project= filterById('Project', project)
#         if(len(project) == 0):
#             print('Project with the specified id does not exist')
#         else:
#             employees= getEmployeeForProject(project[0])
#             for employee in employees:
#                 print(employee[0:4])
#     elif len(project)==1:
#         employees= getEmployeeForProject(project[0][0])
#         for employee in employees:
#                 print(employee[0:4])
#     else:
#         print('Project with the specified name does not exist')
def getProjectsofEmployee(employee_id):
    sql= '''select * from Project join Project_Employee on Project.id=Project_Employee.project_id where employee_id= ?'''
    data= cur.execute(sql,(employee_id,)).fetchall()
    return data
    
def displayProjectsOfEmployee():
    employee= getEmployee()
    print("Selected Employee: "+ str(employee))
    projects= getProjectsofEmployee(employee[0])
    if len(projects)!=0:
        for project in projects:
            print(project[0:4])
    else:
        print('This employee does not have projects yet.')
    
def delete(tableName):
    if tableName== 'Employee':
        employee=getEmployee()
        sql= '''delete from Project_Employee where employee_id= ?'''
        cur.execute(sql, (employee[0],))
        con.commit()
        sql2='''delete from Employee where id= ?'''
        cur.execute(sql, (employee[0],))
        con.commit()
        print("Deleted Employee: "+ str(employee))
    else:
        project= getProject()
        id= project[0]
        print(id)
        sql= '''delete from Project_Employee where project_id= ?'''
        cur.execute(sql, (id,))
        con.commit()
        sql2='''delete from Project where id= ?'''
        cur.execute(sql, (id,))
        con.commit()
        print("Deleted Project: "+ str(project))
        

In [5]:
import sqlite3
con= None
cur= None
def main():
    initialSetup()
    while True:
        choice= getMenuChoice()
        if choice ==1:
            createEmployee()
        elif choice==2:
            createProject()
        elif choice== 3:
            try:
                addEmployeeToProject()
            except Exception as error:
                print(repr(error))
        elif choice==4:
            displayAll("Employee")
        elif choice==5:
            displayAll("Project")
        elif choice==6:
            try:
                displayProjectMembers()
            except Exception as error:
                print(repr(error))
        elif choice==7:
            try:
                displayProjectsOfEmployee()
            except Exception as error:
                print(repr(error))
        elif choice==8:
            delete('Employee')
        elif choice==9:
            delete('Project')
        elif choice== 0:
            con.close()
            break
        

In [6]:
# Fix Delete (Cascade delete)

In [7]:
main()

*************Menu*************
1. Create Employee
2. Create Project
3. Add Employee to the project
4. Display All Employees
5. Display All Projects
6. Display Project Members
7. Display Projects of an employee
8. Delete an employee 
9. Delete a Project
0. Quit
******************************
Enter your choice:9
Enter name of the Project(partial name also works): Project
%Project%
(3, 'Project Tracker', 'Create Users, add projects, manage them', 'high')
(4, 'Project Tracker', 'Add Projects , employees, manage them', 'high')
There are multiple data values that matches with the project name. Please select the id from the list
Enter Project id3
3
Deleted Project: (3, 'Project Tracker', 'Create Users, add projects, manage them', 'high')
*************Menu*************
1. Create Employee
2. Create Project
3. Add Employee to the project
4. Display All Employees
5. Display All Projects
6. Display Project Members
7. Display Projects of an employee
8. Delete an employee 
9. Delete a Project
0. Qui